In [17]:
from pathlib import Path


path_spec_data=Path.cwd().parent.parent/"spec_data/benchmark_spec"
# path_benchmark_data to store benchmark results
path_benchmark_data=Path.cwd().parent.parent/"benchmark_for_test"
path_ms2=Path.cwd().parent.parent/"spec_data/35_random_export_ms2"

path_spec_data.mkdir(parents=True, exist_ok=True)
path_benchmark_data.mkdir(parents=True, exist_ok=True)

In [ ]:
### Only record build and update

original_library_size = [
    # 1_000, 10_000, 100_000,
    1_000_000, 
    # 2_000_000,
    # 10_000_000, 
    # 100_000_000
    
    
    ] 
num_per_group=120_000_000
cache_list_threshold=1_000_000
charge_type = [
    # -1,
                1
                ]
search_type="hybrid"
added_size=100
steps=["build", 
       "update", 
       ]

script_path_dynamic='19_dynamic_entropy_search_library_scale_hybrid_fast_update_mode_every_step.py'
script_path_flash='23_flash_entropy_search_library_scale_hybrid_every_step.py'

In [19]:
# Collect the files
import re
import os
pattern_pos=r"spectra-charge_1_batch_\d+\.mgf"
pattern_neg=r"spectra-charge_-1_batch_\d+\.mgf"
files=os.listdir(Path.cwd().parent.parent/"spec_data/benchmark_spec")
batch_mgf_pos=[]
for file in files:
    if re.match(pattern_pos, file):
        pos_num=file[23:-4]
        batch_mgf_pos.append(pos_num)


batch_mgf_neg=[]
for file in files:
    if re.match(pattern_neg, file):
        neg_num=file[24:-4]
        batch_mgf_neg.append(neg_num)

print(batch_mgf_pos)
print(batch_mgf_neg)

['29', '92', '68', '10', '91', '5', '67', '47', '14', '96']
['56', '32', '67', '70', '87', '98', '54', '75', '88', '21']


In [20]:
import subprocess
import pickle
import os
import time
import numpy as np
import msgpack
import shutil
from typing import Union

def run_usrbintime_by_arguments(
          arguments:list[str], 
          if_output:bool=False, 
          output_memory_file:Union[str,Path]=None, 
          output_time_file:Union[str, Path]=None):
    
    # arguments: script_path, str(charge), step
    command=["/usr/bin/time","-v","python"] + arguments

    if if_output: # Output to files as record
        with open(output_memory_file, "w") as f1, open(output_time_file, "w") as f2:
            subprocess.run(command, stderr=f1, stdout=f2, cwd=Path.cwd(), env=os.environ.copy())

    else: # Output is not needed
         
        subprocess.run(command, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL, cwd=Path.cwd(), env=os.environ.copy())
        
    return


for library_size in original_library_size:
    for charge in charge_type:
        for step in steps:

            if step=="build":
                # Assign the reference files for build, generate the spectra files list
                if 0 < library_size < 1_000_000:
                    spectra_files=[path_spec_data/f"spectra-charge_{charge}-number_{library_size}.pkl"]
                    spectra_mgf_list=[path_spec_data/f"spectra-charge_{charge}-number_{library_size}.mgf"]

                elif 1_000_000<=library_size<100_000_000:
                    file_num=library_size//1_000_000
                    if charge==-1:
                        random_num=batch_mgf_neg
                        
                    elif charge==1:
                        random_num=batch_mgf_pos
                        
                    random_for_use=np.random.choice(random_num, file_num, replace=False)  
                    spectra_files=[]
                    spectra_mgf_list=[]
                    for i in random_for_use:
                        spectra_files.append(path_ms2/f"charge_{charge}/batch_{i}.bin")
                        spectra_mgf_list.append(path_spec_data/f"spectra-charge_{charge}_batch_{i}.mgf")
                
                elif library_size == 100_000_000:
                    spectra_files=[]
                    for i in range(100):
                        spectra_files.append(path_ms2/f"charge_{charge}/batch_{i}.bin")
                        

                # Remove old index of Dynamic Entropy Search
                path_comparison_dynamic_data=Path.cwd().parent.parent/f"comparison_data/dynamic/charge-{charge}"
                if path_comparison_dynamic_data.exists():
                    shutil.rmtree(path_comparison_dynamic_data)

                # Remove old index of Flash Entropy Search
                path_comparison_flash_data=Path.cwd().parent.parent/f"comparison_data/flash/charge-{charge}"
                if path_comparison_flash_data.exists():
                    shutil.rmtree(path_comparison_flash_data)

                # Remove old index of MASST+
                path_masstp=Path.cwd().parent/"library/MASSTplus/build/masst_plus/tools/"
                path_masstp_lib=path_masstp/f"library-{charge}"
                if path_masstp_lib.exists():
                    shutil.rmtree(path_masstp_lib)

            elif step=="update":
                
                insert_spectra_pkl=[path_spec_data/f"insert_spectra-charge_{charge}-number_{added_size}.pkl"]
                insert_spectra_mgf=[path_spec_data/f"insert_spectra-charge_{charge}-number_{added_size}.mgf"]


            ### DynamicEntropySearch ###
            output_memory_file=path_benchmark_data/f"dynamic_fast_update_{charge}_{added_size}_added_to_{library_size}_memory_usage_{step}_step_{search_type}_options_library_size.txt"
            output_time_file=path_benchmark_data/f"dynamic_fast_update_{charge}_{added_size}_added_to_{library_size}_compare_time_{step}_step_{search_type}_options_library_size.txt"
            if step=="build":
                arguments=[script_path_dynamic, str(charge), str(num_per_group), str(cache_list_threshold)] + spectra_files + [step]
            elif step=="update":
                arguments=[script_path_dynamic, str(charge), str(num_per_group), str(cache_list_threshold)] + insert_spectra_pkl + [step]
            run_usrbintime_by_arguments(arguments=arguments, 
                                        if_output=True, 
                                        output_memory_file=output_memory_file,
                                        output_time_file=output_time_file)

            if library_size <= 5_000_000:
                ### FlashEntropySearch ###
                output_memory_file=path_benchmark_data/f"flash_{charge}_{added_size}_added_to_{library_size}_memory_usage_{step}_step_{search_type}_options_library_size.txt"
                output_time_file=path_benchmark_data/f"flash_{charge}_{added_size}_added_to_{library_size}_compare_time_{step}_step_{search_type}_options_library_size.txt"
                if step=="build":
                    arguments=[script_path_flash, str(charge)] + spectra_files + [step]
                elif step=="update":
                    arguments=[script_path_flash, str(charge)] + spectra_files + insert_spectra_pkl + [step]
                    
                run_usrbintime_by_arguments(arguments=arguments,
                                            if_output=True,
                                            output_memory_file=output_memory_file,
                                            output_time_file=output_time_file)
                

            if library_size <= 5_000_000:  
                ### MASST+ ###
                # Write the reference files into a text
                my_lib_list=Path.cwd().parent/"library/MASSTplus/my_library_list.txt"
                if step=="build":
                    mgf_list=spectra_mgf_list
                elif step=="update":
                    mgf_list=insert_spectra_mgf
                    
                with open(my_lib_list,"w") as f:
                    for mgf in mgf_list:
                        f.write(str(mgf))
                        f.write("\n")
                command_str=f"./load {my_lib_list} -r -l {path_masstp_lib}"


                with open(path_benchmark_data / f"masstplus_{charge}_{added_size}_added_to_{library_size}_memory_usage_{step}_step_{search_type}_options_library_size.txt", "w") as f1,\
                    open(path_benchmark_data / f"masstplus_{charge}_{added_size}_added_to_{library_size}_compare_time_{step}_step_{search_type}_options_library_size.txt", "w") as f2:
                    start_time=time.time()
                    result=subprocess.run(
                            ["/usr/bin/time", "-v", "sh", "-c", command_str],
                            stderr=f1,
                            stdout=subprocess.PIPE,
                            env=os.environ.copy(),
                            text=True,
                            cwd=path_masstp
                        )
                    elapsed_time=time.time()-start_time
                    f2.write(str(elapsed_time))
